# Load data
cats = pickle.load(open('./all_cats.pickle', 'rb'))
for cnt, cat in enumerate(cats):
    ind = np.isfinite(cat['eps'])
    lambda_e.extend( cat['lambda_r'][ind])
    eps.extend(cat['eps'][ind])

    lambda_c.extend( cat['lambda_r'][ind])
    stellarmass.extend( cat['mstar'][ind])
    reff.extend( cat['reff'][ind])
    ids.extend( cat['id'][ind] + cnt*10000) # annotation!
    d2t.extend( cat['d2t'][ind])

    rank.extend( 100*(np.argsort(cat['mstar'][ind])/sum(ind) ))

In [1]:
import utils.sampling as smp
import matplotlib.pyplot as plt
import tree
import pickle
import tree.halomodule as hmo
import numpy as np
from analysis.misc import load_cat
import scipy.stats
import tree.ctutils as ctu
#from analysis.evol_lambda_HM import MainPrg
import draw
import load
from utils import match
from analysis.all_plot_modules import *


def extract_main_tree(t, idx, fatherID, fatherMass):
    t_now = t[idx]
    nstep = t_now["nstep"]
    nouts = [nstep]
    atree = np.zeros(nstep + 1, dtype=t.dtype)
    atree[0]=t_now
    
    for i in range(1, nstep + 1):
        try:
            #print(t["flist_index"][idx])
            #id_father = fatherID[t["flist_index"][idx]]
            id_father = fatherID[idx]
            #print(id_father)
            #print(len(id_father))
            if len(id_father) > 1:
                #mass_father = fatherMass[t["flist_index"][idx]]
                mass_father = fatherMass[idx]
                #print(mass_father)
                id_father = id_father[np.argmax(mass_father)]
                ind_father = id_father[id_father > 0] -1
                
                nstep -= 1
                t_father = t[np.where(t["nstep"] == nstep)[0]][ind_father]
                idx = t_father["idx"]
                #print(idx)
                atree[i]=t_father
                nouts.append(nstep)
            else:
                break
        except:
            break
    return atree


def find_closest(A, target):
    #A must be sorted
    idx = A.searchsorted(target)
    idx = np.clip(idx, 1, len(A)-1)
    left = A[idx-1]
    right = A[idx]
    idx -= target - left < right - target
    return idx


def nout2nstep(data, nout):
    return data["nstep"][np.where(data["nout"] == nout)]

def nstep2nout(data, nstep):
    try:
        len(nstep)
        import utils.match as mtc
        ind = mtc.match_list_ind(data["nstep"], nstep)
    except:
        ind = np.where(data["nstep"] == nstep)[0]
    return data["nout"][ind]

def zred2nout(data, nout):
    return data["nstep"][np.where(data["nout"] == nout)]

class MainPrg():
    def __init__(self, idx, nnza):
        """
            Separate tree data and lambda catalog data. 
            
            early snapshot first.
        """
        self.root_idx = idx
        self.nsteps = nnza["nstep"] # truncate later.
        #self.idxs = np.zeros(len(nnza), dtype=int)
        #self.ids = np.zeros(len(nnza), dtype=int)
        self.nouts = nnza["nout"]
        self.zreds = nnza["zred"]
        self.aexps = 1/(1+self.zreds)       
        
    def initialize_data(self, cat, force=False):
        if hasattr(self, "data"):
            if not force:
                print("self.data already exists. use force=True to re-initialize it.")
                pass
        else:
            self.data=np.zeros(len(self.nsteps),
                                  dtype=cat.dtype)

    def set_data(self, cat, nout):
        ind = np.where(cat["tree_root_id"] == self.root_idx)[0]
        if len(ind) == 1:
            #self.data
            inout = np.where(self.nouts == nout)[0]
            if len(inout) == 1:
                self.data[inout] = cat[ind]
                #self.id[inout] = cat[ind]["id"]
    
    def fill_missing_data(self):
        assert (self.ids[0] != 0)
        # position angles cannot be linearly interpolated.
        # skip.
        #
        # position and velocity are also not that linear..
        # but let me just interpolate them.
        # 
        # excluded=["lambda_arr2"]
        filled_fields = ["eps", "epsh", "epsq", "lambda_12kpc",
                         "lambda_arr", "lambda_arrh",
                         "lambda_r","lambda_r12kpc",
                         "lambda_r2","lambda_rh","mgas","mrj","mstar",
                         "reff","reff2","rgal","rgal2","rscale_lambda",
                         "sfr","sma","smah","smaq","smi","smih","smiq","ssfr",
                         "vxc", "vyc", "vzc", "xc", "yc", "zc"]

        i_good_max = max(np.where(self.data["reff"] > 0)[0])
        i_bad = np.where(self.data['idx'] == 0)[0]
        i_bad = i_bad[i_bad < i_good_max]
        if len(i_bad) > 0:
            for field in filled_fields:
                # do not modify index and id fields.
                arr = self.data[field] # it's a view.

                for i_b in i_bad:
                    # neighbouring array might also be empty. Search for closest valid element.
                    # left point
                    i_l = i_b - 1 
                    while(i_l in i_bad):
                        i_l = i_l - 1 
                    # right point
                    i_r = i_b + 1 
                    while(i_r in i_bad):
                        i_r = i_r + 1 
                    arr[i_b] = (arr[i_b -1] + arr[i_b +1])/2.
    
    def truncate(self):
        imax = np.where(self.data["lambda_r"] > 0)[0] + 1
        self.nsteps = self.nsteps[:imax]
        self.nouts = self.nouts[:imax]
        self.zreds = self.zreds[:imax]
        self.aexps = self.aexps[:imax]

In [2]:
import pickle
import matplotlib.pyplot as plt 
import numpy as np

Depending Environments

In [12]:
from analysis.misc import load_cat
from analysis.all_plot_modules import *
from MajorMinorAccretion_module import *

wdir = './Horizon-AGN/Horizon-noAGN/'
#nout_fi = 782
nout_fi=323

nnza = np.genfromtxt(wdir + "nout_nstep_zred_aexp.txt",
                     dtype=[("nout", int),
                            ("nstep", int),
                            ("zred", float),
                            ("aexp", float)])

load_init = False
if load_init:
    mpgs = pickle.load(open(wdir + "MPGS_init.pickle", "rb"))
else:
    mpgs=[]
    #samples = [0,1,2,3,4,6,7,8,9,10,11,12,13,15,16,17,18,19,20]
    samples = [0,1,2,3,4,6,10,11,12,13,15,16,17,18]
    for sample in samples:
        # initialize mpgs
        ss = str(sample)
        f_cat = load_cat(wdir + 'lambda_results/' + ss + '/catalog' + str(nout_fi) + ss +'.pickle')
        root_idx_all = f_cat['idx'][f_cat["idx"] > 0].astype(int) # why idx are float???
        for i, idx in enumerate(root_idx_all):
            #atree = extract_main_tree(tt, idx, fatherID, fatherMass)
            mpgs.append(MainPrg(idx, nnza))

        # assign lambda measurement data
        for nout in nnza["nout"][:1]:
            cat = load_cat(wdir + 'lambda_results/' + ss + '/catalog' + str(nout) + ss +'.pickle')
            #print(nout)
            for gal in mpgs:
                if nout == nout_fi:
                    gal.initialize_data(cat, force=True)
                gal.set_data(cat, nout)
        
    for gal in mpgs:
        gal.ids = gal.data["id"]
        gal.idxs = gal.data["idx"]
        #self.ids = np.zeros(len(nnza), dtype=int)
        

In [13]:
from tree import halomodule as hmo
allgal = hmo.Halo(nout=nout_fi, is_gal=True, base=wdir)

import load
info = load.info.Info(nout=nout_fi, base=wdir)

galidlist = []
for gal in mpgs:
    #print(gal.ids)
    #plt.plot(np.log10(gal.data["mstar"]))
    galidlist.append(gal.ids[0])

In [14]:
xall = allgal.data["x"]
yall = allgal.data["y"]
zall = allgal.data["z"]

bins = np.linspace(0, 1, 20)

xbin = np.digitize(xall, bins)
ybin = np.digitize(yall, bins)
zbin = np.digitize(zall, bins)

dist_cut = 5/info.pboxsize

d5 = []
N5 = []
for idgal in galidlist:
#for igal in range(len(allgal.data)):
    igal = np.where(allgal.data["id"] == idgal)[0]
    x_this = allgal.data["x"][igal]
    y_this = allgal.data["y"][igal]
    z_this = allgal.data["z"][igal]
    
    # get subsample to speed up the code
    xb_this = xbin[igal]
    yb_this = ybin[igal]
    zb_this = zbin[igal]
    
    first_candidates = allgal.data[(np.abs(xbin - xb_this) < 2) \
                                  * (np.abs(ybin - yb_this) < 2)\
                                  * (np.abs(zbin - zb_this) < 2)]
    
    dist = np.sqrt(np.square(first_candidates["x"] - x_this) + 
                   np.square(first_candidates["y"] - y_this) + 
                   np.square(first_candidates["z"] - z_this))
    #print(len(dist))
    N5.append(sum(dist < dist_cut))
    d5.append(np.sort(dist)[4] * info.pboxsize)

In [15]:
d5_80 = d5[np.argsort(d5)[np.ceil(0.8 * len(d5)).astype(int)]]
i_isol = np.where(d5 >= d5_80)[0]
isolated = [mpgs[i] for i in i_isol]

d5_20 = d5[np.argsort(d5)[np.ceil(0.2 * len(d5)).astype(int)]]
i_dense = np.where(d5 <= d5_20)[0]
dense = [mpgs[i] for i in i_dense]

eps_a = np.array([gal.data["eps"][0] for gal in mpgs])
lambda_a = np.array([gal.data["lambda_r"][0] for gal in mpgs])

eps_i = np.array([gal.data["eps"][0] for gal in isolated])
lambda_i = np.array([gal.data["lambda_r"][0] for gal in isolated])

eps_d = np.array([gal.data["eps"][0] for gal in dense])
lambda_d = np.array([gal.data["lambda_r"][0] for gal in dense])

In [16]:
print(len(eps_a), len(eps_d), len(eps_i))

8874 1776 1774


In [17]:
atlas = np.genfromtxt('/home/hoseung/Work/data/ATLAS3D/Emsellem2011_Atlas3D_Paper3_TableB1.txt',
                      skip_header=12,
                      usecols=(2,7))
twocolors=['#4c72b0', '#c44e52']


do_plot(eps_a, lambda_a, atlas,
        do_scatter=False,
        contour_label=False,
        surf = False,
        img_scale = 2.0,
        twocolors=twocolors,
        den_cmap = "YlGnBu_r",
        d_alpha=1.0,
        levels=None,#np.linspace(0.02, 1.0, 19),
        fname_vs_e = "./figs/lambda_vs_e_allEnvNH")

do_plot(eps_i, lambda_i, atlas,
        do_scatter=False,
        contour_label=False,
        surf = False,
        img_scale = 2.0,
        twocolors=twocolors,
        den_cmap = "YlGnBu_r",
        d_alpha=1.0,
        levels=None,#np.linspace(0.02, 1.0, 19),
        fname_vs_e = "./figs/lambda_vs_e_isolNH")

do_plot(eps_d, lambda_d, atlas,
        do_scatter=False,
        contour_label=False,
        surf = False,
        img_scale = 2.0,
        twocolors=twocolors,
        den_cmap = "YlGnBu_r",
        d_alpha=1.0,
        levels=None,#np.linspace(0.02, 1.0, 19),
        fname_vs_e = "./figs/lambda_vs_e_denseNH")

In [72]:
# Discrete levels
do_scatter=True
contour_label=False
surf = False


import itertools
for levels in [np.linspace(0.01, 1.0, 19), None]:
    for combination in itertools.product(*[(True, False)]*3):
        do_scatter, contour_label, surf = combination
        do_plot(x,y, 
                do_scatter=do_scatter,
                contour_label=contour_label,
                surf = surf,
                img_scale = 2.0,
                twocolors=twocolors,
                den_cmap = "PuBu",
                levels=levels,
                fname_vs_e = "./figs/lambda_vs_e_z0")